#### CKY Parsing Algorithm

The `CKY` algorithm is a `dynamic programming algorithm` for efficiently creating all possible parse trees for a given sentence and context-free grammar. It requires the grammar to be in `Chomsky Normal Form (CNF)`, i.e. each production is either of the form $A \to B C$ or $A \to a$, where $A,B,C \in N$ and $a \in \Sigma$.

We will first define our grammar (borrowed from Figure 17.8, Jurafsky-Martin textbook), convert it to CNF and then implement the CKY parser.

In [1]:
import nltk

In [5]:
# define the grammar
grammar = nltk.CFG.fromstring("""
    S -> NP VP | Aux NP VP | VP 
    NP -> Det Nom | PropN | Pronoun 
    Nom -> Noun | Nom Noun | Nom PP
    VP -> Verb | Verb NP | Verb NP PP | Verb PP | VP PP
    PP -> Prep NP
    Det -> 'that' | 'this' | 'the' | 'a' 
    Noun -> 'book' | 'flight' | 'meal' | 'money'
    Verb -> 'book' | 'include' | 'prefer'
    Pronoun -> 'I' | 'she' | 'me'
    PropN -> 'Houston' | 'NWA'
    Aux -> 'does'
    Prep -> 'from' | 'to' | 'on' | 'near' | 'through'
""")

To convert this grammar to CNF, we will first binarize all the productions:

`S -> Aux NP VP` becomes `S -> X1 VP` and `X1 -> Aux NP`

`VP -> Verb NP PP` becomes `VP -> X2 PP` and `X2 -> Verb NP`


Then we convert the unit production `S -> VP` into the following:

S -> VP 
=> S -> Verb | Verb NP | X2 PP | Verb PP | VP PP 
=> `S -> 'book' | 'include' | 'prefer' | Verb NP | X2 PP | Verb PP | VP PP`


In [6]:
# grammar in CNF
grammar_CNF = nltk.CFG.fromstring("""
    S -> NP VP | X1 VP | 'book' | 'include' | 'prefer' | Verb NP | X2 PP | Verb PP | VP PP 
    X1 -> Aux NP
    NP -> Det Nom | PropN | Pronoun 
    Nom -> Noun | Nom Noun | Nom PP
    VP -> Verb | Verb NP | X2 PP | Verb PP | VP PP
    X2 -> Verb NP                              
    PP -> Prep NP
    Det -> 'that' | 'this' | 'the' | 'a' 
    Noun -> 'book' | 'flight' | 'meal' | 'money'
    Verb -> 'book' | 'include' | 'prefer'
    Pronoun -> 'I' | 'she' | 'me'
    PropN -> 'Houston' | 'NWA'
    Aux -> 'does'
    Prep -> 'from' | 'to' | 'on' | 'near' | 'through'
""")

In [7]:
# show all the productions, terminal and non-terminal symbols
terminals = set()
non_terminals = set()
for production in grammar_CNF.productions():
    print(production)
    print(f"\tLHS: {production.lhs()}")
    print(f"\tRHS: {production.rhs()}")
    non_terminals.add(production.lhs())
    for symbol in production.rhs():
        if nltk.grammar.is_terminal(symbol):
            terminals.add(symbol)
        else:
            non_terminals.add(symbol)

print(f"Non-terminals: {non_terminals}")
print(f"Terminals: {terminals}")

S -> NP VP
	LHS: S
	RHS: (NP, VP)
S -> X1 VP
	LHS: S
	RHS: (X1, VP)
S -> 'book'
	LHS: S
	RHS: ('book',)
S -> 'include'
	LHS: S
	RHS: ('include',)
S -> 'prefer'
	LHS: S
	RHS: ('prefer',)
S -> Verb NP
	LHS: S
	RHS: (Verb, NP)
S -> X2 PP
	LHS: S
	RHS: (X2, PP)
S -> Verb PP
	LHS: S
	RHS: (Verb, PP)
S -> VP PP
	LHS: S
	RHS: (VP, PP)
X1 -> Aux NP
	LHS: X1
	RHS: (Aux, NP)
NP -> Det Nom
	LHS: NP
	RHS: (Det, Nom)
NP -> PropN
	LHS: NP
	RHS: (PropN,)
NP -> Pronoun
	LHS: NP
	RHS: (Pronoun,)
Nom -> Noun
	LHS: Nom
	RHS: (Noun,)
Nom -> Nom Noun
	LHS: Nom
	RHS: (Nom, Noun)
Nom -> Nom PP
	LHS: Nom
	RHS: (Nom, PP)
VP -> Verb
	LHS: VP
	RHS: (Verb,)
VP -> Verb NP
	LHS: VP
	RHS: (Verb, NP)
VP -> X2 PP
	LHS: VP
	RHS: (X2, PP)
VP -> Verb PP
	LHS: VP
	RHS: (Verb, PP)
VP -> VP PP
	LHS: VP
	RHS: (VP, PP)
X2 -> Verb NP
	LHS: X2
	RHS: (Verb, NP)
PP -> Prep NP
	LHS: PP
	RHS: (Prep, NP)
Det -> 'that'
	LHS: Det
	RHS: ('that',)
Det -> 'this'
	LHS: Det
	RHS: ('this',)
Det -> 'the'
	LHS: Det
	RHS: ('the',)
Det -> 'a'
	

CKY is a dynamic programming algorithm for finding valid parse trees for a given sentence. It requires the grammar to be in CFN so that each production rule is binary. It has two steps: 1) `Recognition` and 2) `Backtracking`

The `recognition` involves considering each possible span in a sentence and labelling it with an appropriate constituent from the grammar. First, we define `fenceposts` which are positions between adjacent words in the sentence. For a sentence with `n` words, position `0` is to the left of the first word and position `n` is to the right of the last word. Then we define `span [i,j]` as the words between positions `i` and `j`,  e.g. for the sentence, `0 Book 1 that 2 flight 3`, span `[1,3]` denotes `that flight`. 

Then for each word in the sentence (in order starting from first), we consider all spans that have right fencepost fixed at the end of that word and left fencepost which starts to the left of that word and advances one step at a time until reaching position `0`, so e.g. for the word `that`, we will first consider the span `[1,2]`, label it (since this is a single word, i.e. a terminal symbol, we assign the label as the left hand side of the production that generates this terminal, in this case the production is `Det -> that` so the label is `Det`) and then we label `[0,2]`. 

Equivalently, we can construct an `(n+1) x (n+1)` matrix whose `(i,j)th` element contains the label for span `[i,j]`, then fill the upper triangular portion of this matrix, column by column, starting from bottom of the column to top. Furtehrmore, since each span can be thought of as being composed by two constituents (because the productions are binary), we also need to consider all possible splits at position `k` where `i<k<j`. Because of the way we are filling column by column from bottom to top, the labels for the the two constituents of each splits will already have been computed. Say the labels for a split are `B C`, then if we find a production `A -> B C`, then we append the label `A` to cell `(i,j)`, i.e. we can have multiple labels inside each cell. In addition to storing the label, we also store pointers to the cells containing the two consituents. 

What makes this a dynamic programming algorithm is this systematic way of filling the matrix involves combining smaller consitutents into bigger ones. Then a valid parse exists if we end up with a label of `S` in the `(0,n)th` entry in the matrix which is the last one to be filled. To obtain the parse trees, we then have to perform `backtracing` to recursively retreive the consituentes via their pointers.